### 决策树

In [1]:
# 导入该用的一些库

import numpy as np
import pandas as pd
from pandas import Series,DataFrame
from sklearn import tree


先用pandas来看看数据长什么样吧。

In [7]:
data_train = pd.read_table('lenses.txt',sep='\t',header=None)
data_train

,0,1,2,3,4
0,young,myope,no,reduced,no lenses
1,young,myope,no,normal,soft
2,young,myope,yes,reduced,no lenses
3,young,myope,yes,normal,hard
4,young,hyper,no,reduced,no lenses
5,young,hyper,no,normal,soft
6,young,hyper,yes,reduced,no lenses
7,young,hyper,yes,normal,hard
8,pre,myope,no,reduced,no lenses
9,pre,myope,no,normal,soft


当然也可以自己写加载数据的函数:

In [8]:
def loadData(filename):
    fr = open(filename)
    
    # 读取生成列表
    lenses = [inst.strip().split('\t') for inst in fr.readlines()]
    
    # 构成标签列表
    lensesLabels = ['age','prescript','astigmatic','tearRate']
    
    return lenses,lensesLabels

In [9]:
lenses,lensesLabels = loadData('lenses.txt')

In [10]:
lenses

[['young', 'myope', 'no', 'reduced', 'no lenses'],
 ['young', 'myope', 'no', 'normal', 'soft'],
 ['young', 'myope', 'yes', 'reduced', 'no lenses'],
 ['young', 'myope', 'yes', 'normal', 'hard'],
 ['young', 'hyper', 'no', 'reduced', 'no lenses'],
 ['young', 'hyper', 'no', 'normal', 'soft'],
 ['young', 'hyper', 'yes', 'reduced', 'no lenses'],
 ['young', 'hyper', 'yes', 'normal', 'hard'],
 ['pre', 'myope', 'no', 'reduced', 'no lenses'],
 ['pre', 'myope', 'no', 'normal', 'soft'],
 ['pre', 'myope', 'yes', 'reduced', 'no lenses'],
 ['pre', 'myope', 'yes', 'normal', 'hard'],
 ['pre', 'hyper', 'no', 'reduced', 'no lenses'],
 ['pre', 'hyper', 'no', 'normal', 'soft'],
 ['pre', 'hyper', 'yes', 'reduced', 'no lenses'],
 ['pre', 'hyper', 'yes', 'normal', 'no lenses'],
 ['presbyopic', 'myope', 'no', 'reduced', 'no lenses'],
 ['presbyopic', 'myope', 'no', 'normal', 'no lenses'],
 ['presbyopic', 'myope', 'yes', 'reduced', 'no lenses'],
 ['presbyopic', 'myope', 'yes', 'normal', 'hard'],
 ['presbyopic', 

先来按照书上的构建决策树的方法来。用ID3算法来划分数据集，也就是需要先计算数据集的熵。

In [13]:
from math import log

# 计算香农熵
def calcShannonEnt(dataSet):
    # 获取数据集的大小
    numEntries = len(dataSet)
    labelCounts = {}
    
    # 遍历数据集
    for featVec in dataSet:
        # 抽取数据中的类型
        currentLabel = featVec[-1]
        
        # 计算该类型在数据集中出现的次数
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
        
    # 初始化香农熵
    shannonEnt = 0.0
    
    # 遍历类型字典计算香农熵
    for key in labelCounts:
        # 根据香农熵公式来计算，先求概率（频率），然后求经验熵
        prob = float(labelCounts[key])/numEntries
        shannonEnt -= prob * log(prob,2)
    return shannonEnt

先来计算看看整个数据集的熵:

In [14]:
calcShannonEnt(lenses)

1.3260875253642983

能够求得熵之后，就可以实现划分数据集的函数了。

In [15]:
'''
划分数据集的函数
参数：数据集，划分数据集的特征，特征值
'''
def splitDataSet(dataSet,axis,value):
    retDataSet = []
    
    # 遍历数据集
    for featVec in dataSet:
        # 判断当前划分的特征对应值是否需要返回
        if featVec[axis] == value:
            # 将当前处理的数据列表featVec进行重构组合
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
            
    return retDataSet

In [16]:
# 获取最好的数据划分方式
def chooseBestFeatureToSplit(dataSet):
    # 用来划分的特征数量，去掉最后一列的分类属性
    numFeatures = len(dataSet[0]) - 1
    
    # 计算数据集的香农熵
    baseEntropy = calcShannonEnt(dataSet)
    
    # 初始化
    bestInfoGain = 0.0; bestFeature = -1
    
    # 遍历所有特征
    for i in range(numFeatures):
        # 提取该特征对应的所有值
        featList = [example[i] for example in dataSet]
        
        # 去重
        uniqueVals = set(featList)
        newEntropy = 0.0
        
        # 循环使用该特征值作为参数进行数据集划分尝试
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet,i,value)
            
            # 对划分后的数据集计算香农熵，并进行求和操作（条件熵）
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
            
        # 计算信息增益
        infoGain = baseEntropy - newEntropy
        
        # 获取最好的信息增益
        if(infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature

测试测试以上函数

In [17]:
lensesLabels

['age', 'prescript', 'astigmatic', 'tearRate']

In [18]:
chooseBestFeatureToSplit(lenses)

3

也就是说认为第3个特征是最好的(是以0开始数的)

In [19]:
lensesLabels[3]

'tearRate'

知道选择哪个特征是好特征之后，就可以进行决策树的构建了。

In [20]:
# 构建决策树
def createTree(dataSet,labels):
    # 提取数据集中的所有分类标签
    classList = [example[-1] for example in dataSet]
    
    # 数据集的类别完全相同时则停止继续对数据集进行划分，并返回分类标签
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    
    # 如果所有的特征值都已经遍历结束
    if len(dataSet[0]) == 1:
        # 计算出现频次最多的分类标签并返回
        classCount = {}
        for vote in classList:
            if vote not in classCount.keys():
                classCount[vote] = 0
            classsCount[vote] += 1
        sortedClassCount = sorted(classCount.iteritems(),key=operator.itemgetter(1),
                                 reverse=True)
        return sortedClassCount[0][0]
    
    # 获取最好的划分特征
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    
    # 以当前数据集中获取的最好的特征初始化树
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    
    # 提取该特征对应的所有特征值
    featValues = [example[bestFeat] for example in dataSet]
    
    # 去重，需要根据唯一的特征进行划分
    uniqueVals = set(featValues)
    
    # 划分数据集，创建树的分支，将不同特征值对应的数据集进行递归划分
    for value in uniqueVals:
        subLabels = labels[:]
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet,bestFeat,value),subLabels)
    return myTree

测试算法，是以字典的形式展出的

In [21]:
tree = createTree(lenses,lensesLabels)

In [22]:
tree

{'tearRate': {'normal': {'astigmatic': {'no': {'age': {'pre': 'soft',
      'presbyopic': {'prescript': {'hyper': 'soft', 'myope': 'no lenses'}},
      'young': 'soft'}},
    'yes': {'prescript': {'hyper': {'age': {'pre': 'no lenses',
        'presbyopic': 'no lenses',
        'young': 'hard'}},
      'myope': 'hard'}}}},
  'reduced': 'no lenses'}}

如果需要画出来，需要matplotlib

In [ ]:
import matplotlib.pyplot as plt

def plotMidText(cntrPt,parentPt,txtString):
    xMid = (parentPt[0]-cntrPt[0])/2.0 + cnt